In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import pandas as pd
import os
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Select data and metadata

In [ ]:
data_file = "/content/drive/MyDrive/Master - A/test.csv"
data_file2 = "//content/drive/MyDrive/Master - A/test2.csv"
df = pd.read_csv(data_file)
df2 = pd.read_csv(data_file2)
df = pd.concat([df, df2])
import os
import pandas as pd
import numpy as np
from PIL import Image


dataset_path = "//content/drive/MyDrive/Master - A/Ostateczny"
img_size = (224, 224)
x = os.listdir(dataset_path)
selected_classes = x

data = []
X=[]
for class_name in selected_classes:
    class_path = os.path.join(dataset_path, class_name)
    if not os.path.isdir(class_path):
        continue
    for img_file in os.listdir(class_path):
        if not img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
            continue

        img_path = os.path.join(class_path, img_file)
        filtered_row = df[df["image_path"] == img_file]

        if filtered_row.empty:
            continue 

        try:
            img = Image.open(img_path).convert('RGB')
            img = img.resize(img_size)
            img_array = np.array(img) / 255.0  
            X.append(img_array)

            row_data = {
                "image_path": img_file,
                "image_array": img_array,
                "label": class_name,
                "Substrate": filtered_row["Substrate"].values[0],
                "Habitat": filtered_row["Habitat"].values[0],
                "Latitude": filtered_row["Latitude"].values[0],
                "Longitude": filtered_row["Longitude"].values[0],
                "CoorUncert": filtered_row["CoorUncert"].values[0],
                "locality": filtered_row["locality"].values[0]
            }

            data.append(row_data)

        except Exception as e:
            print(f"Błąd przy {img_path}: {e}")

df_all = pd.DataFrame(data)


In [3]:
features = df_all[["Habitat", "Substrate", "Latitude", "Longitude", "CoorUncert"]]

le_habitat = LabelEncoder()
le_substrate = LabelEncoder()

features["Habitat_enc"] = le_habitat.fit_transform(features["Habitat"])
features["Substrate_enc"] = le_substrate.fit_transform(features["Substrate"])
X_meta = features[["Habitat_enc", "Substrate_enc", "Latitude", "Longitude", "CoorUncert"]].values
X_meta.shape

<ipython-input-3-b641cc0550e7>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features["Habitat_enc"] = le_habitat.fit_transform(features["Habitat"])
<ipython-input-3-b641cc0550e7>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features["Substrate_enc"] = le_substrate.fit_transform(features["Substrate"])


(8397, 5)

In [4]:
#X = df_all["image_array"]
# X = np.stack(df_all["image_array"].values)
y = df_all["label"]
le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_cat = to_categorical(y_encoded)
y_all = np.array(y_cat)
X = np.array(X)
X_img_train, X_img_test, X_meta_train, X_meta_test, y_train_enc, y_test_enc = train_test_split(
    X, X_meta, y_all, test_size=0.2, random_state=42, stratify=y_all
)

### Save pickle

In [ ]:
import pickle
import os

save_dir = "/content/drive/MyDrive/Master - A/"
os.makedirs(save_dir, exist_ok=True)

datasets = {
    "X_img_train": X_img_train,
    "X_img_test": X_img_test,
    "X_meta_train": X_meta_train,
    "X_meta_test": X_meta_test,
    "y_train_enc": y_train_enc,
    "y_test_enc": y_test_enc,
    "label_encoder": le,
}

for name, data in datasets.items():
    file_path = os.path.join(save_dir, f"{name}.pkl")
    with open(file_path, "wb") as f:
        pickle.dump(data, f)
    print(f"✔ Zapisano: {file_path}")


NameError: name 'X_img_train' is not defined

In [6]:
del X, y, df_all

In [7]:
del X_meta

In [17]:
df_all = pd.read_csv("/content/drive/MyDrive/Master - A/3_fig/Dane.csv")

### Save pickle

In [ ]:
import pickle
load_dir = "/content/drive/MyDrive/Master - A/"

dataset_names = [
    "X_img_train",
    "X_img_test",
    "X_meta_train",
    "X_meta_test",
    "y_train_enc",
    "y_test_enc",
    "label_encoder"
]

loaded_data = {}

for name in dataset_names:
    file_path = os.path.join(load_dir, f"{name}.pkl")
    with open(file_path, "rb") as f:
        loaded_data[name] = pickle.load(f)
    print(f" Wczytano: {file_path}")

X_img_train = loaded_data["X_img_train"]
X_img_test = loaded_data["X_img_test"]
X_meta_train = loaded_data["X_meta_train"]
X_meta_test = loaded_data["X_meta_test"]
y_train_enc = loaded_data["y_train_enc"]
y_test_enc = loaded_data["y_test_enc"]
le = loaded_data["label_encoder"]

✔ Wczytano: /content/drive/MyDrive/Master - A/X_img_train.pkl
✔ Wczytano: /content/drive/MyDrive/Master - A/X_img_test.pkl
✔ Wczytano: /content/drive/MyDrive/Master - A/X_meta_train.pkl
✔ Wczytano: /content/drive/MyDrive/Master - A/X_meta_test.pkl
✔ Wczytano: /content/drive/MyDrive/Master - A/y_train_enc.pkl
✔ Wczytano: /content/drive/MyDrive/Master - A/y_test_enc.pkl
✔ Wczytano: /content/drive/MyDrive/Master - A/label_encoder.pkl


In [ ]:
col_means = np.nanmean(X_meta_test, axis=0)
inds = np.where(np.isnan(X_meta_test))
X_meta_test[inds] = np.take(col_means, inds[1])

col_means = np.nanmean(X_meta_train, axis=0)
inds = np.where(np.isnan(X_meta_train))
X_meta_train[inds] = np.take(col_means, inds[1])

X_img_train_flat = X_img_train.reshape(X_img_train.shape[0], -1)
X_img_test_flat = X_img_test.reshape(X_img_test.shape[0], -1)
X_train_combined = np.concatenate([X_img_train_flat, X_meta_train], axis=1)
X_test_combined = np.concatenate([X_img_test_flat, X_meta_test], axis=1)

In [7]:
X_train_new, X_val, y_train_new, y_val = train_test_split(
    X_train_combined, y_train_enc, test_size=0.1, random_state=42, stratify=y_train_enc
)

In [8]:
X_val.shape

(672, 150533)

In [ ]:
model_classes = [
    ('KNN_D', KNeighborsClassifier()),
    ('DecisionTree_D', DecisionTreeClassifier()),
    ('RandomForest_D', RandomForestClassifier()),
    ('LogisticRegression_D', LogisticRegression())
]

param_grid = {
    'KNN_D': [{'n_neighbors': k} for k in [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]],
    'DecisionTree_D': [{'max_depth': d} for d in [5, 10, 15, 20, 25, 30, None]],
    'RandomForest_D': [{'n_estimators': n, 'max_depth': d, 'max_features': 'sqrt'} for n in [50, 100, 1000] for d in [10, 20]],
}

def classical_sim(X_train, X_test, y_train, y_test, param=120):
    results = []

    for name, model in model_classes:
        print(f"\n{name} - training...")

        for param_set in param_grid.get(name, []):
            print(f"Testing with parameters: {param_set}")

            start_time = time.time()

            model.set_params(**param_set)

            print(f"\n{name} - training...")

            model.fit(X_train_combined, y_train_enc)
            end_time = time.time()
            training_time = end_time - start_time

            preds = model.predict(X_test_combined)
            acc_preview = accuracy_score(y_test_enc, preds)

            y_prob = model.predict(X_test_combined)
            y_pred = np.argmax(y_prob, axis=1)
            #y_pred = y_prob
            y_true = np.argmax(y_test_enc, axis=1)

            accuracy = accuracy_score(y_true, y_pred)
            report = classification_report(y_true, y_pred)

            with open(f"/content/drive/MyDrive/Master - A/3_fig/{param}_{name}_results.txt", "w") as f:
                f.write(f"Model: {name}\n")
                f.write(f"Training time: {training_time:.2f} sec\n")
                f.write(f"Accuracy: {accuracy:.4f}\n\n")
                f.write("Classification Report:\n")
                f.write(report)

            conf_matrix = confusion_matrix(y_true, y_pred)
            plt.figure(figsize=(8, 6))
            sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=le.classes_, yticklabels=le.classes_)
            plt.title("Confusion Matrix")
            plt.xlabel("Predicted")
            plt.ylabel("Actual")
            plt.tight_layout()
            plt.savefig(f"/content/drive/MyDrive/Master - A/3_fig/{param}_{name}_matrix.png")
            plt.close()

            fpr, tpr, _ = roc_curve(y_true, y_prob[:, 1]) if y_prob.shape[1] == 2 else (None, None, None)
            if fpr is not None:
                roc_auc = auc(fpr, tpr)
                plt.figure(figsize=(8, 6))
                plt.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.2f})', lw=2)
                plt.plot([0, 1], [0, 1], 'k--')
                plt.xlabel('False Positive Rate')
                plt.ylabel('True Positive Rate')
                plt.legend()
                plt.tight_layout()
                plt.savefig(f"/content/drive/MyDrive/Master - A/3_fig/{param}_{name}_roc_curve.png")
                plt.close()

            results.append((name, accuracy))

            del preds
            gc.collect()

        with open(f"/content/drive/MyDrive/Master - A/3_fig/{param}_final_results.csv", mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Model", "Accuracy"])
            for name, acc in results:
                writer.writerow([name, f"{acc:.4f}"])

        print("\nFinal Results:")
        for name, acc in results:
            print(f"{name}: {acc:.4f}")




In [11]:
import time
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc
import csv

In [ ]:
X

In [21]:
classical_sim(X_train_combined, X_test_combined, y_train_enc, y_test_enc, param = "june")



KNN_D - training...
Testing with parameters: {'n_neighbors': 10}

KNN_D - training...


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Testing with parameters: {'n_neighbors': 11}

KNN_D - training...


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Testing with parameters: {'n_neighbors': 12}

KNN_D - training...


...

Testing with parameters: {'n_estimators': 1000, 'max_depth': 10, 'max_features': 'sqrt'}

RandomForest_D - training...


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Testing with parameters: {'n_estimators': 1000, 'max_depth': 20, 'max_features': 'sqrt'}

RandomForest_D - training...


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Final Results:
KNN_D: 0.1536
KNN_D: 0.1643
KNN_D: 0.1458
KNN_D: 0.1595
KNN_D: 0.1446
KNN_D: 0.1554
KNN_D: 0.1452
KNN_D: 0.1530
KNN_D: 0.1440
KNN_D: 0.1512
KNN_D: 0.1423
KNN_D: 0.1476
KNN_D: 0.1399
KNN_D: 0.1435
KNN_D: 0.1375
KNN_D: 0.1435
DecisionTree_D: 0.2833
DecisionTree_D: 0.2946
DecisionTree_D: 0.3042
DecisionTree_D: 0.3327
DecisionTree_D: 0.3196
DecisionTree_D: 0.3452
DecisionTree_D: 0.3274
RandomForest_D: 0.1262
RandomForest_D: 0.1381
RandomForest_D: 0.1286
RandomForest_D: 0.1339
RandomForest_D: 0.1256
RandomForest_D: 0.1327

LogisticRegression_D - training...

Final Results:
KNN_D: 0.1536
KNN_D: 0.1643
KNN_D: 0.1458
KNN_D: 0.1595
KNN_D: 0.1446
KNN_D: 0.1554
KNN_D: 0.1452
KNN_D: 0.1530
KNN_D: 0.1440
KNN_D: 0.1512
KNN_D: 0.1423
KNN_D: 0.1476
KNN_D: 0.1399
KNN_D: 0.1435
KNN_D: 0.1375
KNN_D: 0.1435
DecisionTree_D: 0.2833
DecisionTree_D: 0.2946
DecisionTree_D: 0.3042
DecisionTree_D: 0.3327
DecisionTree_D: 0.3196
DecisionTree_D: 0.3452
DecisionTree_D: 0.3274
RandomForest_D: 0.1262
R

In [ ]:
classical_sim(X_train_combined, X_test_combined, y_train_enc, y_test_enc, param = "june2")


In [ ]:
model_classes = [
    ('KNN_D', KNeighborsClassifier()),
    #('DecisionTree_D', DecisionTreeClassifier()),
    #('RandomForest_D', RandomForestClassifier()),
    #('LogisticRegression_D', LogisticRegression())
]

param_grid = {
    'KNN_D': [{'n_neighbors': k} for k in [3, 4, 5, 6,7, 8,9, 10]],
    #'DecisionTree_D': [{'max_depth': d} for d in [5, 10, 15, 20, 25, 30, None]],
    #'RandomForest_D': [{'n_estimators': n, 'max_depth': d, 'max_features': 'sqrt'} for n in [50, 100, 1000] for d in [10, 20]],
}

In [ ]:
import time
import gc
import csv
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (
    accuracy_score, classification_report,
    confusion_matrix, roc_curve, auc
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

def classical_sim_grid(X_val, y_val, X_train, X_test, y_train, y_test, param="grid"):
    results = []

    model_classes = {
        'KNN_D': (KNeighborsClassifier(), {
            'n_neighbors': list(range(3, 26))
        }),
        'DecisionTree_D': (DecisionTreeClassifier(), {
            'max_depth': [5, 10, 15, 20, 25, 30, None]
        }),
        'RandomForest_D': (RandomForestClassifier(), {
            'n_estimators': [50, 100, 1000],
            'max_depth': [10, 20],
            'max_features': ['sqrt']
        }),
        'LogisticRegression_D': (LogisticRegression(max_iter=1000), {})  # domyślnie, brak gridu
    }

    for name, (model, param_grid) in model_classes.items():
        print(f"\n{name} - Grid Search...")

        grid_search = GridSearchCV(
            estimator=model,
            param_grid=param_grid,
            scoring='accuracy',
            cv=3,
            n_jobs=-1,
            verbose=1
        )

        start_time = time.time()
        grid_search.fit(X_val, y_val)
        training_time = time.time() - start_time

        best_model = grid_search.best_estimator_
        best_params = grid_search.best_params_
        print(f"Best params: {best_params}")

        start_time = time.time()
        model.set_params(**best_params)
        model.fit(X_train, y_train)
        end_time = time.time()
        training_time = end_time - start_time

        preds = model.predict(X_test_combined)
        acc_preview = accuracy_score(y_test_enc, preds)

        y_prob = model.predict(X_test_combined)
        #y_pred = np.argmax(y_prob, axis=1)
        y_pred = y_prob
        y_true = y_test_enc
        #y_true = np.argmax(y_test_enc, axis=1)

        accuracy = accuracy_score(y_true, y_pred)
        report = classification_report(y_true, y_pred)

        filename_prefix = f"/content/drive/MyDrive/Master - A/2_fig/{param}_{name}"
        with open(f"{filename_prefix}_results.txt", "w") as f:
            f.write(f"Model: {name}\n")
            f.write(f"Best Parameters: {best_params}\n")
            f.write(f"Training time: {training_time:.2f} sec\n")
            f.write(f"Accuracy: {accuracy:.4f}\n\n")
            f.write("Classification Report:\n")
            f.write(report)

        conf_matrix = confusion_matrix(y_test, preds)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.title(f"{name} - Confusion Matrix")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.tight_layout()
        plt.savefig(f"{filename_prefix}_matrix.png")
        plt.close()

        try:
            if len(np.unique(y_test)) == 2:
                y_score = best_model.predict_proba(X_test)[:, 1]
                fpr, tpr, _ = roc_curve(y_test, y_score)
                roc_auc = auc(fpr, tpr)

                plt.figure(figsize=(8, 6))
                plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC (AUC = {roc_auc:.2f})')
                plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
                plt.xlabel('False Positive Rate')
                plt.ylabel('True Positive Rate')
                plt.title('ROC Curve')
                plt.legend(loc="lower right")
                plt.tight_layout()
                plt.savefig(f"{filename_prefix}_roc_curve.png")
                plt.close()
        except Exception as e:
            print(f"⚠️ ROC failed: {e}")

        results.append((name, best_params, accuracy))

        del grid_search, best_model, preds, conf_matrix
        gc.collect()

    with open(f"/content/drive/MyDrive/Master - A/2_fig/{param}_final_results.csv", mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Model", "Best Parameters", "Accuracy"])
        for name, best_params, acc in results:
            writer.writerow([name, str(best_params), f"{acc:.4f}"])

    print("\nFinal Results:")
    for name, best_params, acc in results:
        print(f"{name} with best parameters {best_params}: {acc:.4f}")



In [23]:
classical_sim(X_train_combined, X_test_combined, y_train_enc, y_test_enc, param = "june2")



KNN_D - training...
Testing with parameters: {'n_neighbors': 3}

KNN_D - training...
Testing with parameters: {'n_neighbors': 4}

KNN_D - training...
Testing with parameters: {'n_neighbors': 5}

KNN_D - training...
Testing with parameters: {'n_neighbors': 6}

KNN_D - training...
Testing with parameters: {'n_neighbors': 7}

KNN_D - training...
Testing with parameters: {'n_neighbors': 8}

KNN_D - training...


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Testing with parameters: {'n_neighbors': 9}

KNN_D - training...


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Testing with parameters: {'n_neighbors': 10}

KNN_D - training...


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Final Results:
KNN_D: 0.2018
KNN_D: 0.1643
KNN_D: 0.1869
KNN_D: 0.1613
KNN_D: 0.1810
KNN_D: 0.1554
KNN_D: 0.1708
KNN_D: 0.1536


#GRID SEARCH

In [22]:
y_train_enc = np.argmax(y_train_enc, axis=1) if len(y_train_enc.shape) > 1 else y_train_enc
y_test_enc = np.argmax(y_test_enc, axis=1) if len(y_test_enc.shape) > 1 else y_test_enc
y_val = np.argmax(y_val, axis=1) if len(y_val.shape) > 1 else y_val
classical_sim_grid(X_val, y_val, X_train_combined, X_test_combined, y_train_enc, y_test_enc, param = "grid_juneeee")



KNN_D - Grid Search...
Fitting 3 folds for each of 23 candidates, totalling 69 fits
Best params: {'n_neighbors': 20}

DecisionTree_D - Grid Search...
Fitting 3 folds for each of 7 candidates, totalling 21 fits
Best params: {'max_depth': 5}


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



RandomForest_D - Grid Search...
Fitting 3 folds for each of 6 candidates, totalling 18 fits
Best params: {'max_depth': 20, 'max_features': 'sqrt', 'n_estimators': 1000}

LogisticRegression_D - Grid Search...
Fitting 3 folds for each of 1 candidates, totalling 3 fits


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Best params: {}


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



Final Results:
KNN_D with best parameters {'n_neighbors': 20}: 0.2554
DecisionTree_D with best parameters {'max_depth': 5}: 0.3375
RandomForest_D with best parameters {'max_depth': 20, 'max_features': 'sqrt', 'n_estimators': 1000}: 0.4137
LogisticRegression_D with best parameters {}: 0.4071


## CNNN



In [ ]:
import os
import random
import pickle
import numpy as np
random.seed(42)
from sklearn.metrics import classification_report, accuracy_score,confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.applications import (
    VGG16, VGG19, ResNet50, InceptionV3, Xception,
    MobileNet, MobileNetV2, MobileNetV3Large, DenseNet121, DenseNet169, NASNetMobile,
    EfficientNetB0, EfficientNetB1, EfficientNetB2, EfficientNetB7
)
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:


model_checkpoint_callback = ModelCheckpoint(
    filepath='best_cnn_model.keras', 
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

In [ ]:
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=6,
    restore_best_weights=True
)

IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 70


model_classes = [
    InceptionV3, Xception, MobileNet,MobileNetV2,
    DenseNet121, VGG16
]

model_names = [
    "InceptionV3", "Xception", "MobileNet", "MobileNetV2",
    "DenseNet121", "VGG16"
]

import time

In [ ]:
def my_cnn(input_shape=(224, 224, 3), num_classes=10):
    model = models.Sequential()

    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))

    model.add(layers.Dense(num_classes, activation='softmax'))

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model


In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import load_model

import time
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, auc
from tensorflow.keras import models, layers
import pickle
import pandas as pd

def cnn_sim(x_train, x_test, y_train, y_test, param):
    results = []

    for model_class, name in zip(model_classes, model_names):
        print(f"\n{name}")

        base_model = model_class(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
        base_model.trainable = False  

        model = models.Sequential([
            base_model,
            layers.GlobalAveragePooling2D(),
            layers.Dense(128, activation='relu'),
            layers.Dropout(0.5),
            layers.Dense(len(le.classes_), activation='softmax')  
        ])

        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

        start_time = time.time()
        history = model.fit(
            x_train, y_train,
            epochs=70,
            batch_size=64,
            validation_split=0.2,
            verbose=2,
            callbacks=[early_stopping]
        )
        end_time = time.time()
        training_time = end_time - start_time
        print(f"Training time: {training_time:.2f} seconds")

        loss, acc = model.evaluate(x_test, y_test)
        print(f"{name} - Test accuracy: {acc:.4f}")

        y_prob = model.predict(x_test)
        y_pred = np.argmax(y_prob, axis=1)
        y_true = np.argmax(y_test, axis=1)

        accuracy = accuracy_score(y_true, y_pred)
        print(f"Accuracy: {accuracy:.4f}")

        with open(f"/content/drive/MyDrive/Master - A/3_fig/{param}_{name}_results.txt", "w") as f:
            f.write(f"Model: {name}\n")
            f.write(f"Training time: {training_time:.2f} sec\n")
            f.write(f"Accuracy: {acc:.4f}\n\n")
            f.write("Classification Report:\n")

        conf_matrix = confusion_matrix(y_true, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=le.classes_, yticklabels=le.classes_)
        plt.title("Confusion Matrix")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.tight_layout()
        plt.savefig(f"/content/drive/MyDrive/Master - A/3_fig/{param}_{name}_matrix.png")
        plt.close()

        model.save(f'/content/drive/MyDrive/Master - A/3_fig/{param}_{name}_model.h5')

        plt.figure(figsize=(8, 6))
        plt.plot(history.history['loss'], label='Train Loss')
        plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.xlabel('Epochs', fontsize=18)
        plt.ylabel('Loss', fontsize=18)
        plt.xticks(fontsize=18)
        plt.yticks(fontsize=18)
        plt.tight_layout()
        plt.legend()
        plt.savefig(f"/content/drive/MyDrive/Master - A/3_fig/{param}_{name}_train_val_loss.png")
        plt.close()

        plt.figure(figsize=(8, 6))
        plt.plot(history.history['accuracy'], label='Train Accuracy')
        plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
        plt.xlabel('Epochs', fontsize=18)
        plt.ylabel('Accuracy', fontsize=18)
        plt.xticks(fontsize=18)
        plt.yticks(fontsize=18)
        plt.tight_layout()
        plt.legend()
        plt.savefig(f"/content/drive/MyDrive/Master - A/3_fig/{param}_{name}_train_val_accuracy.png")
        plt.close()

        fpr, tpr, _ = roc_curve(y_true, y_prob[:, 1])  
        roc_auc = auc(fpr, tpr)
        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        plt.xlabel('False Positive Rate', fontsize=18)
        plt.ylabel('True Positive Rate', fontsize=18)
        plt.xticks(fontsize=18)
        plt.yticks(fontsize=18)
        plt.tight_layout()
        plt.legend(loc="lower right")
        plt.savefig(f"/content/drive/MyDrive/Master - A/3_fig/{param}_{name}_roc_curve.png")
        plt.close()

        results.append({
            'Model': name,
            'Test Accuracy': acc,
            'Training Time (s)': training_time
        })

    model = my_cnn(num_classes=len(le.classes_))
    name = "my"

    start_time = time.time()
    history = model.fit(x_train, y_train, epochs=70, batch_size=32,
                        validation_split=0.2, verbose=1, callbacks=[early_stopping])
    end_time = time.time()
    training_time = end_time - start_time
    print(f"Training time: {training_time:.2f} seconds")

    test_loss, test_acc = model.evaluate(x_test, y_test)
    print(f"Test accuracy: {test_acc:.4f}")

    y_prob = model.predict(x_test)
    y_pred = np.argmax(y_prob, axis=1)
    y_true = np.argmax(y_test, axis=1)

    conf_matrix = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=le.classes_, yticklabels=le.classes_)
    plt.title("Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.tight_layout()
    plt.savefig(f"/content/drive/MyDrive/Master - A/3_fig/{param}_{name}_matrix.png")
    plt.close()

    model.save(f'/content/drive/MyDrive/Master - A/3_fig/{param}_{name}_model.h5')

    file_path = f"/content/drive/MyDrive/Master - A/3_fig/{param}_{name}_history.pkl"
    with open(file_path, 'wb') as f:
        pickle.dump(history.history, f)

    plt.figure(figsize=(8, 6))
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs', fontsize=18)
    plt.ylabel('Loss', fontsize=18)
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)
    plt.tight_layout()
    plt.legend()
    plt.savefig(f"/content/drive/MyDrive/Master - A/3_fig/{param}_{name}_train_val_loss.png")
    plt.close()

    plt.figure(figsize=(8, 6))
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epochs', fontsize=18)
    plt.ylabel('Accuracy', fontsize=18)
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)
    plt.tight_layout()
    plt.legend()
    plt.savefig(f"/content/drive/MyDrive/Master - A/3_fig/{param}_{name}_train_val_accuracy.png")
    plt.close()

    results.append({
        'Model': name,
        'Test Accuracy': test_acc,
        'Training Time (s)': training_time
    })

    results_df = pd.DataFrame(results)
    results_df.to_csv(f'/content/drive/MyDrive/Master - A/3_fig/{param}_model_training_results.csv', index=False)
    print(results_df)


### Bez meta

In [ ]:
param = "bez_meta"

results = []

for model_class, name in zip(model_classes, model_names):
    print(f"\n{name}")

    base_model = model_class(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
    base_model.trainable = False  

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(len(le.classes_), activation='softmax') 
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    start_time = time.time()
    history = model.fit(
        X_img_train, y_train_enc,
        epochs=70,
        batch_size=64,
        validation_split=0.2,
        verbose=2,
        callbacks=[early_stopping]
    )
    end_time = time.time()
    training_time = end_time - start_time
    print(f"Training time: {training_time:.2f} seconds")

    loss, acc = model.evaluate(X_img_test, y_test_enc)
    print(f"{name} - Test accuracy: {acc:.4f}")

    y_prob = model.predict(X_img_test)
    y_pred = np.argmax(y_prob, axis=1)
    y_true = np.argmax(y_test_enc, axis=1)
    print(y_true)
    print(y_pred)
    accuracy = accuracy_score(y_true, y_pred)
    print(f"Accuracy: {accuracy:.4f}")

    with open(f"/content/drive/MyDrive/Master - A/3_fig/{param}_{name}_results.txt", "w") as f:
        f.write(f"Model: {name}\n")
        f.write(f"Training time: {training_time:.2f} sec\n")
        f.write(f"Accuracy: {acc:.4f}\n\n")
        f.write("Classification Report:\n")

    conf_matrix = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=le.classes_, yticklabels=le.classes_)
    plt.title("Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.tight_layout()
    plt.savefig(f"/content/drive/MyDrive/Master - A/3_fig/{param}_{name}_matrix.png")
    plt.close()

    model.save(f'/content/drive/MyDrive/Master - A/3_fig/{param}_{name}_model.h5')


    file_path = f"/content/drive/MyDrive/Master - A/3_fig/{param}_{name}_history.pkl"
    with open(file_path, 'wb') as f:
        pickle.dump(history.history, f)

    plt.figure(figsize=(8, 6))
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs', fontsize=18)
    plt.ylabel('Loss', fontsize=18)
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)
    plt.tight_layout()
    plt.legend()
    plt.savefig(f"/content/drive/MyDrive/Master - A/3_fig/{param}_{name}_train_val_loss.png")
    plt.close()

    plt.figure(figsize=(8, 6))
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epochs', fontsize=18)
    plt.ylabel('Accuracy', fontsize=18)
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)
    plt.tight_layout()
    plt.legend()
    plt.savefig(f"/content/drive/MyDrive/Master - A/3_fig/{param}_{name}_train_val_accuracy.png")
    plt.close()

    results.append({
        'Model': name,
        'Test Accuracy': acc,
          'Training Time (s)': training_time
      })

model = my_cnn(num_classes=len(le.classes_))
name = "my"

start_time = time.time()
history = model.fit(X_img_train, y_train_enc, epochs=70, batch_size=32,
                    validation_split=0.2, verbose=1, callbacks=[early_stopping])
end_time = time.time()
training_time = end_time - start_time
print(f"Training time: {training_time:.2f} seconds")

test_loss, test_acc = model.evaluate(X_img_test, y_test_enc)
print(f"Test accuracy: {test_acc:.4f}")

y_prob = model.predict(X_img_test)
y_pred = np.argmax(y_prob, axis=1)
y_true = np.argmax(y_test_enc, axis=1)

conf_matrix = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=le.classes_, yticklabels=le.classes_)
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.tight_layout()
plt.savefig(f"/content/drive/MyDrive/Master - A/3_fig/{param}_{name}_matrix.png")
plt.close()

model.save(f'/content/drive/MyDrive/Master - A/3_fig/{param}_{name}_model.h5')

file_path = f"/content/drive/MyDrive/Master - A/3_fig/{param}_{name}_history.pkl"
with open(file_path, 'wb') as f:
    pickle.dump(history.history, f)

plt.figure(figsize=(8, 6))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs', fontsize=18)
plt.ylabel('Loss', fontsize=18)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.tight_layout()
plt.legend()
plt.savefig(f"/content/drive/MyDrive/Master - A/3_fig/{param}_{name}_train_val_loss.png")
plt.close()

plt.figure(figsize=(8, 6))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs', fontsize=18)
plt.ylabel('Accuracy', fontsize=18)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.tight_layout()
plt.legend()
plt.savefig(f"/content/drive/MyDrive/Master - A/3_fig/{param}_{name}_train_val_accuracy.png")
plt.close()

results.append({
    'Model': name,
    'Test Accuracy': test_acc,
    'Training Time (s)': training_time
})

results_df = pd.DataFrame(results)
results_df.to_csv(f'/content/drive/MyDrive/Master - A/3_fig/{param}_model_training_results.csv', index=False)
print(results_df)


InceptionV3
Epoch 1/70
84/84 - 30s - 351ms/step - accuracy: 0.4018 - loss: 1.6580 - val_accuracy: 0.6064 - val_loss: 1.1429
Epoch 2/70
84/84 - 5s - 59ms/step - accuracy: 0.5522 - loss: 1.1888 - val_accuracy: 0.6384 - val_loss: 0.9978
Epoch 3/70
84/84 - 5s - 59ms/step - accuracy: 0.6066 - loss: 1.0241 - val_accuracy: 0.6823 - val_loss: 0.8744
Epoch 4/70
84/84 - 5s - 59ms/step - accuracy: 0.6289 - loss: 0.9429 - val_accuracy: 0.6935 - val_loss: 0.8291
Epoch 5/70
84/84 - 5s - 59ms/step - accuracy: 0.6771 - loss: 0.8661 - val_accuracy: 0.6815 - val_loss: 0.8214
Epoch 6/70
84/84 - 5s - 59ms/step - accuracy: 0.6752 - loss: 0.8416 - val_accuracy: 0.7217 - val_loss: 0.7637
Epoch 7/70
84/84 - 5s - 57ms/step - accuracy: 0.7067 - loss: 0.7590 - val_accuracy: 0.7039 - val_loss: 0.7817
Epoch 8/70
84/84 - 5s - 57ms/step - accuracy: 0.7182 - loss: 0.7403 - val_accuracy: 0.7158 - val_loss: 0.7703
Epoch 9/70
84/84 - 5s - 60ms/step - accuracy: 0.7268 - loss: 0.7180 - val_accuracy: 0.7068 - val_loss: 0.


Xception
83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
Epoch 1/70
84/84 - 44s - 526ms/step - accuracy: 0.5038 - loss: 1.3252 - val_accuracy: 0.6503 - val_loss: 0.9351
Epoch 2/70
84/84 - 10s - 119ms/step - accuracy: 0.6438 - loss: 0.9176 - val_accuracy: 0.6920 - val_loss: 0.8059
Epoch 3/70
84/84 - 10s - 119ms/step - accuracy: 0.6948 - loss: 0.7948 - val_accuracy: 0.7202 - val_loss: 0.7403
Epoch 4/70
84/84 - 10s - 119ms/step - accuracy: 0.7294 - loss: 0.7135 - val_accuracy: 0.7307 - val_loss: 0.7053
Epoch 5/70
84/84 - 10s - 120ms/step - accuracy: 0.7523 - loss: 0.6484 - val_accuracy: 0.7277 - val_loss: 0.7039
Epoch 6/70
84/84 - 10s - 119ms/step - accuracy: 0.7675 - loss: 0.6084 - val_accuracy: 0.7299 - val_loss: 0.6822
Epoch 7/70
84/84 - 10s - 120ms/step - accuracy: 0.7765 - loss: 0.5654 - val_accuracy: 0.7426 - val_loss: 0.6771
Epoch 8/70
84/84 - 10s - 119ms/step - accuracy: 0.8042 - loss: 0.5227 - val_accuracy: 0.7321 - val_loss: 0.6831
Epoch 9/70
84/84 - 10s - 120ms/step - accur


MobileNet
17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Epoch 1/70
84/84 - 25s - 301ms/step - accuracy: 0.4204 - loss: 1.5815 - val_accuracy: 0.6682 - val_loss: 0.9584
Epoch 2/70
84/84 - 4s - 45ms/step - accuracy: 0.6121 - loss: 1.0116 - val_accuracy: 0.7083 - val_loss: 0.7879
Epoch 3/70
84/84 - 4s - 44ms/step - accuracy: 0.6728 - loss: 0.8482 - val_accuracy: 0.7336 - val_loss: 0.7083
Epoch 4/70
84/84 - 4s - 44ms/step - accuracy: 0.7238 - loss: 0.7452 - val_accuracy: 0.7388 - val_loss: 0.6892
Epoch 5/70
84/84 - 4s - 43ms/step - accuracy: 0.7327 - loss: 0.6939 - val_accuracy: 0.7463 - val_loss: 0.6705
Epoch 6/70
84/84 - 4s - 44ms/step - accuracy: 0.7670 - loss: 0.6173 - val_accuracy: 0.7634 - val_loss: 0.6438
Epoch 7/70
84/84 - 4s - 44ms/step - accuracy: 0.7782 - loss: 0.5882 - val_accuracy: 0.7545 - val_loss: 0.6435
Epoch 8/70
84/84 - 4s - 44ms/step - accuracy: 0.7992 - loss: 0.5290 - val_accuracy: 0.7693 - val_loss: 0.6168
Epoch 9/70
84/84 - 4s - 45ms/step - accuracy: 0.8079 - l


MobileNetV2
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Epoch 1/70
84/84 - 31s - 372ms/step - accuracy: 0.4768 - loss: 1.4437 - val_accuracy: 0.6778 - val_loss: 0.8943
Epoch 2/70
84/84 - 4s - 51ms/step - accuracy: 0.6538 - loss: 0.9200 - val_accuracy: 0.7240 - val_loss: 0.7297
Epoch 3/70
84/84 - 4s - 51ms/step - accuracy: 0.7059 - loss: 0.7713 - val_accuracy: 0.7217 - val_loss: 0.6973
Epoch 4/70
84/84 - 4s - 51ms/step - accuracy: 0.7391 - loss: 0.6730 - val_accuracy: 0.7381 - val_loss: 0.6930
Epoch 5/70
84/84 - 4s - 51ms/step - accuracy: 0.7694 - loss: 0.5938 - val_accuracy: 0.7463 - val_loss: 0.6480
Epoch 6/70
84/84 - 4s - 52ms/step - accuracy: 0.7927 - loss: 0.5460 - val_accuracy: 0.7515 - val_loss: 0.6279
Epoch 7/70
84/84 - 4s - 51ms/step - accuracy: 0.8117 - loss: 0.4967 - val_accuracy: 0.7626 - val_loss: 0.6209
Epoch 8/70
84/84 - 4s - 50ms/step - accuracy: 0.8310 - loss: 0.4544 - val_accuracy: 0.7552 - val_loss: 0.6219
Epoch 9/70
84/84 - 4s - 50ms/step - accuracy: 0.8385 - l


DenseNet121
29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Epoch 1/70
84/84 - 69s - 821ms/step - accuracy: 0.4275 - loss: 1.5962 - val_accuracy: 0.6801 - val_loss: 0.9429
Epoch 2/70
84/84 - 9s - 103ms/step - accuracy: 0.6415 - loss: 0.9703 - val_accuracy: 0.7299 - val_loss: 0.7370
Epoch 3/70
84/84 - 8s - 101ms/step - accuracy: 0.6966 - loss: 0.8144 - val_accuracy: 0.7574 - val_loss: 0.6533
Epoch 4/70
84/84 - 9s - 102ms/step - accuracy: 0.7262 - loss: 0.7227 - val_accuracy: 0.7567 - val_loss: 0.6204
Epoch 5/70
84/84 - 9s - 102ms/step - accuracy: 0.7554 - loss: 0.6559 - val_accuracy: 0.7686 - val_loss: 0.5872
Epoch 6/70
84/84 - 9s - 102ms/step - accuracy: 0.7752 - loss: 0.6078 - val_accuracy: 0.7723 - val_loss: 0.5694
Epoch 7/70
84/84 - 9s - 102ms/step - accuracy: 0.7809 - loss: 0.5838 - val_accuracy: 0.7760 - val_loss: 0.5638
Epoch 8/70
84/84 - 9s - 101ms/step - accuracy: 0.7934 - loss: 0.5544 - val_accuracy: 0.7790 - val_loss: 0.5599
Epoch 9/70
84/84 - 8s - 101ms/step - accuracy: 


VGG16
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Epoch 1/70
84/84 - 57s - 677ms/step - accuracy: 0.2233 - loss: 2.1138 - val_accuracy: 0.3564 - val_loss: 1.8637
Epoch 2/70
84/84 - 15s - 183ms/step - accuracy: 0.3406 - loss: 1.8131 - val_accuracy: 0.3899 - val_loss: 1.6667
Epoch 3/70
84/84 - 15s - 184ms/step - accuracy: 0.3881 - loss: 1.6687 - val_accuracy: 0.4702 - val_loss: 1.5263
Epoch 4/70
84/84 - 15s - 184ms/step - accuracy: 0.4294 - loss: 1.5560 - val_accuracy: 0.5134 - val_loss: 1.4374
Epoch 5/70
84/84 - 15s - 183ms/step - accuracy: 0.4591 - loss: 1.4806 - val_accuracy: 0.5342 - val_loss: 1.3596
Epoch 6/70
84/84 - 15s - 182ms/step - accuracy: 0.4871 - loss: 1.4101 - val_accuracy: 0.5379 - val_loss: 1.3188
Epoch 7/70
84/84 - 15s - 182ms/step - accuracy: 0.5055 - loss: 1.3673 - val_accuracy: 0.5655 - val_loss: 1.2666
Epoch 8/70
84/84 - 15s - 182ms/step - accuracy: 0.5189 - loss: 1.3271 - val_accuracy: 0.5744 - val_loss: 1.2413
Epoch 9/70
84/84 - 15s - 184ms/step - accuracy

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/70
168/168 ━━━━━━━━━━━━━━━━━━━━ 19s 78ms/step - accuracy: 0.1542 - loss: 2.3682 - val_accuracy: 0.3222 - val_loss: 1.8882
Epoch 2/70
168/168 ━━━━━━━━━━━━━━━━━━━━ 7s 40ms/step - accuracy: 0.2971 - loss: 1.8594 - val_accuracy: 0.4033 - val_loss: 1.6009
Epoch 3/70
168/168 ━━━━━━━━━━━━━━━━━━━━ 7s 40ms/step - accuracy: 0.4354 - loss: 1.5446 - val_accuracy: 0.4807 - val_loss: 1.4091
Epoch 4/70
168/168 ━━━━━━━━━━━━━━━━━━━━ 7s 40ms/step - accuracy: 0.5061 - loss: 1.3607 - val_accuracy: 0.5097 - val_loss: 1.3842
Epoch 5/70
168/168 ━━━━━━━━━━━━━━━━━━━━ 7s 40ms/step - accuracy: 0.5899 - loss: 1.1466 - val_accuracy: 0.5260 - val_loss: 1.3313
Epoch 6/70
168/168 ━━━━━━━━━━━━━━━━━━━━ 7s 40ms/step - accuracy: 0.6790 - loss: 0.9180 - val_accuracy: 0.5365 - val_loss: 1.3655
Epoch 7/70
168/168 ━━━━━━━━━━━━━━━━━━━━ 7s 40ms/step - accuracy: 0.7370 - loss: 0.6997 - val_accuracy: 0.5208 - val_loss: 1.5140
Epoch 8/70
168/168 ━━━━━━━━━━━━━━━━━━━━ 7s 40ms/step - accuracy: 0.8019 - loss: 0.5591 - val_acc

         Model  Test Accuracy  Training Time (s)
0  InceptionV3       0.738690         127.865819
1     Xception       0.760119         208.671374
2    MobileNet       0.792857          89.317813
3  MobileNetV2       0.794048          99.906089
4  DenseNet121       0.818452         209.636113
5        VGG16       0.675000         891.858847
6           my       0.551190          91.106546


### Z meta

In [ ]:
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Concatenate, Dropout

def cnn_sim_combined(x_train_img, x_test_img, x_train_meta, x_test_meta, y_train, y_test, param):
    results = []

    for model_class, name in zip(model_classes, model_names):
        print(f"\n{name}")

        base_model = model_class(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
        base_model.trainable = False

        image_input = Input(shape=(IMG_SIZE, IMG_SIZE, 3), name='image_input')
        meta_input = Input(shape=(x_train_meta.shape[1],), name='meta_input')

        x = base_model(image_input, training=False)
        x = GlobalAveragePooling2D()(x)
        x = Dense(128, activation='relu')(x)

        m = Dense(64, activation='relu')(meta_input)

        combined = Concatenate()([x, m])
        combined = Dense(128, activation='relu')(combined)
        combined = Dropout(0.5)(combined)
        output = Dense(len(le.classes_), activation='softmax')(combined)

        model = Model(inputs=[image_input, meta_input], outputs=output)
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

        start_time = time.time()
        history = model.fit(
            [x_train_img, x_train_meta], y_train,
            epochs=70,
            batch_size=64,
            validation_split=0.2,
            verbose=2,
            callbacks=[early_stopping]
        )
        training_time = time.time() - start_time

        loss, acc = model.evaluate([x_test_img, x_test_meta], y_test)
        print(f"{name} - Test accuracy: {acc:.4f}")

        y_prob = model.predict([x_test_img, x_test_meta])
        y_pred = np.argmax(y_prob, axis=1)
        y_true = np.argmax(y_test, axis=1)

        print("\nClassification Report:")
        print(classification_report(y_true, y_pred))

        with open(f"/content/drive/MyDrive/Master - A/3_fig/{param}_{name}_results.txt", "w") as f:
            f.write(f"Model: {name}\n")
            f.write(f"Training time: {training_time:.2f} sec\n")
            f.write(f"Accuracy: {acc:.4f}\n\n")
            f.write("Classification Report:\n")
            f.write(classification_report(y_true, y_pred))

        conf_matrix = confusion_matrix(y_true, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=le.classes_, yticklabels=le.classes_)
        plt.title("Confusion Matrix")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.tight_layout()
        plt.savefig(f"/content/drive/MyDrive/Master - A/3_fig/{param}_{name}_matrix.png")
        plt.close()

        plt.figure(figsize=(8, 6))
        plt.plot(history.history['loss'], label='Train Loss')
        plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
        plt.tight_layout()
        plt.savefig(f"/content/drive/MyDrive/Master - A/3_fig/{param}_{name}_train_val_loss.png")
        plt.close()

        plt.figure(figsize=(8, 6))
        plt.plot(history.history['accuracy'], label='Train Accuracy')
        plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.tight_layout()
        plt.savefig(f"/content/drive/MyDrive/Master - A/3_fig/{param}_{name}_train_val_accuracy.png")
        plt.close()

        fpr, tpr, _ = roc_curve(y_true, y_prob[:, 1]) if y_prob.shape[1] == 2 else (None, None, None)
        if fpr is not None:
            roc_auc = auc(fpr, tpr)
            plt.figure(figsize=(8, 6))
            plt.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.2f})', lw=2)
            plt.plot([0, 1], [0, 1], 'k--')
            plt.xlabel('False Positive Rate')
            plt.ylabel('True Positive Rate')
            plt.legend()
            plt.tight_layout()
            plt.savefig(f"/content/drive/MyDrive/Master - A/3_fig/{param}_{name}_roc_curve.png")
            plt.close()

        model.save(f'/content/drive/MyDrive/Master - A/3_fig/{param}_{name}_model.h5')

        results.append({
            'Model': name,
            'Test Accuracy': acc,
            'Training Time (s)': training_time
        })

    
    results_df = pd.DataFrame(results)
    results_df.to_csv(f'/content/drive/MyDrive/Master - A/3_fig/{param}_model_training_results.csv', index=False)
    print(results_df)


from tensorflow.keras import Input, Model
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Concatenate, Dropout



In [ ]:
X_img_train.shape, X_img_test.shape, X_meta_test.shape, X_meta_train.shape, y_train_enc.shape, y_test_enc.shape

((6717, 224, 224, 3),
 (1680, 224, 224, 3),
 (1680, 2),
 (6717, 2),
 (6717, 10),
 (1680, 10))

In [ ]:
cnn_sim_combined(X_img_train, X_img_test, X_meta_train, X_meta_test, y_train_enc, y_test_enc, "meta")



InceptionV3
Epoch 1/70
84/84 - 34s - 405ms/step - accuracy: 0.4057 - loss: 1.6776 - val_accuracy: 0.6280 - val_loss: 1.0266
Epoch 2/70
84/84 - 5s - 61ms/step - accuracy: 0.5924 - loss: 1.0740 - val_accuracy: 0.6585 - val_loss: 0.8833
Epoch 3/70
84/84 - 5s - 61ms/step - accuracy: 0.6523 - loss: 0.9052 - val_accuracy: 0.7001 - val_loss: 0.7963
Epoch 4/70
84/84 - 5s - 61ms/step - accuracy: 0.7059 - loss: 0.7682 - val_accuracy: 0.7240 - val_loss: 0.7281
Epoch 5/70
84/84 - 5s - 59ms/step - accuracy: 0.7346 - loss: 0.6938 - val_accuracy: 0.7202 - val_loss: 0.7464
Epoch 6/70
84/84 - 5s - 59ms/step - accuracy: 0.7614 - loss: 0.6222 - val_accuracy: 0.7188 - val_loss: 0.7317
Epoch 7/70
84/84 - 5s - 60ms/step - accuracy: 0.7698 - loss: 0.5845 - val_accuracy: 0.7314 - val_loss: 0.7055
Epoch 8/70
84/84 - 5s - 59ms/step - accuracy: 0.7936 - loss: 0.5466 - val_accuracy: 0.7351 - val_loss: 0.7198
Epoch 9/70
84/84 - 5s - 59ms/step - accuracy: 0.8174 - loss: 0.4754 - val_accuracy: 0.7269 - val_loss: 0.


Xception
Epoch 1/70
84/84 - 26s - 310ms/step - accuracy: 0.4541 - loss: 1.4850 - val_accuracy: 0.6548 - val_loss: 0.8977
Epoch 2/70
84/84 - 10s - 119ms/step - accuracy: 0.6348 - loss: 0.9157 - val_accuracy: 0.7031 - val_loss: 0.7474
Epoch 3/70
84/84 - 10s - 120ms/step - accuracy: 0.7061 - loss: 0.7618 - val_accuracy: 0.7113 - val_loss: 0.7240
Epoch 4/70
84/84 - 10s - 119ms/step - accuracy: 0.7366 - loss: 0.6657 - val_accuracy: 0.7158 - val_loss: 0.7016
Epoch 5/70
84/84 - 10s - 120ms/step - accuracy: 0.7724 - loss: 0.5967 - val_accuracy: 0.7426 - val_loss: 0.6645
Epoch 6/70
84/84 - 10s - 119ms/step - accuracy: 0.7919 - loss: 0.5319 - val_accuracy: 0.7336 - val_loss: 0.6536
Epoch 7/70
84/84 - 10s - 119ms/step - accuracy: 0.8042 - loss: 0.4879 - val_accuracy: 0.7448 - val_loss: 0.6289
Epoch 8/70
84/84 - 10s - 118ms/step - accuracy: 0.8358 - loss: 0.4267 - val_accuracy: 0.7426 - val_loss: 0.6732
Epoch 9/70
84/84 - 10s - 119ms/step - accuracy: 0.8487 - loss: 0.3899 - val_accuracy: 0.7440 -


MobileNet
Epoch 1/70
84/84 - 16s - 191ms/step - accuracy: 0.4044 - loss: 1.6576 - val_accuracy: 0.6481 - val_loss: 0.9479
Epoch 2/70
84/84 - 4s - 44ms/step - accuracy: 0.6200 - loss: 0.9813 - val_accuracy: 0.7054 - val_loss: 0.7631
Epoch 3/70
84/84 - 4s - 44ms/step - accuracy: 0.7020 - loss: 0.7901 - val_accuracy: 0.7307 - val_loss: 0.6732
Epoch 4/70
84/84 - 4s - 44ms/step - accuracy: 0.7381 - loss: 0.6816 - val_accuracy: 0.7381 - val_loss: 0.6426
Epoch 5/70
84/84 - 4s - 43ms/step - accuracy: 0.7735 - loss: 0.5859 - val_accuracy: 0.7619 - val_loss: 0.6118
Epoch 6/70
84/84 - 4s - 44ms/step - accuracy: 0.7958 - loss: 0.5290 - val_accuracy: 0.7783 - val_loss: 0.5807
Epoch 7/70
84/84 - 4s - 43ms/step - accuracy: 0.8280 - loss: 0.4611 - val_accuracy: 0.7671 - val_loss: 0.5906
Epoch 8/70
84/84 - 4s - 43ms/step - accuracy: 0.8411 - loss: 0.4206 - val_accuracy: 0.7641 - val_loss: 0.5934
Epoch 9/70
84/84 - 4s - 43ms/step - accuracy: 0.8519 - loss: 0.3882 - val_accuracy: 0.7701 - val_loss: 0.58


MobileNetV2
Epoch 1/70
84/84 - 22s - 261ms/step - accuracy: 0.4673 - loss: 1.4655 - val_accuracy: 0.6927 - val_loss: 0.8334
Epoch 2/70
84/84 - 4s - 51ms/step - accuracy: 0.6588 - loss: 0.8777 - val_accuracy: 0.7135 - val_loss: 0.7338
Epoch 3/70
84/84 - 4s - 51ms/step - accuracy: 0.7316 - loss: 0.6887 - val_accuracy: 0.7433 - val_loss: 0.6460
Epoch 4/70
84/84 - 4s - 51ms/step - accuracy: 0.7783 - loss: 0.5753 - val_accuracy: 0.7567 - val_loss: 0.6105
Epoch 5/70
84/84 - 4s - 49ms/step - accuracy: 0.8092 - loss: 0.5036 - val_accuracy: 0.7463 - val_loss: 0.6356
Epoch 6/70
84/84 - 4s - 50ms/step - accuracy: 0.8278 - loss: 0.4477 - val_accuracy: 0.7641 - val_loss: 0.6185
Epoch 7/70
84/84 - 4s - 51ms/step - accuracy: 0.8593 - loss: 0.3695 - val_accuracy: 0.7641 - val_loss: 0.5987
Epoch 8/70
84/84 - 4s - 50ms/step - accuracy: 0.8612 - loss: 0.3603 - val_accuracy: 0.7560 - val_loss: 0.6365
Epoch 9/70
84/84 - 4s - 50ms/step - accuracy: 0.8863 - loss: 0.2999 - val_accuracy: 0.7619 - val_loss: 0.


DenseNet121
Epoch 1/70
84/84 - 58s - 694ms/step - accuracy: 0.3921 - loss: 1.6989 - val_accuracy: 0.6436 - val_loss: 0.9616
Epoch 2/70
84/84 - 8s - 100ms/step - accuracy: 0.6118 - loss: 0.9834 - val_accuracy: 0.7180 - val_loss: 0.7137
Epoch 3/70
84/84 - 8s - 100ms/step - accuracy: 0.6953 - loss: 0.7704 - val_accuracy: 0.7448 - val_loss: 0.6503
Epoch 4/70
84/84 - 9s - 102ms/step - accuracy: 0.7433 - loss: 0.6671 - val_accuracy: 0.7783 - val_loss: 0.5627
Epoch 5/70
84/84 - 8s - 99ms/step - accuracy: 0.7748 - loss: 0.5804 - val_accuracy: 0.7589 - val_loss: 0.5711
Epoch 6/70
84/84 - 9s - 102ms/step - accuracy: 0.7869 - loss: 0.5439 - val_accuracy: 0.7783 - val_loss: 0.5405
Epoch 7/70
84/84 - 8s - 101ms/step - accuracy: 0.8061 - loss: 0.4895 - val_accuracy: 0.7820 - val_loss: 0.5301
Epoch 8/70
84/84 - 8s - 99ms/step - accuracy: 0.8172 - loss: 0.4666 - val_accuracy: 0.7708 - val_loss: 0.5617
Epoch 9/70
84/84 - 8s - 98ms/step - accuracy: 0.8331 - loss: 0.4399 - val_accuracy: 0.7909 - val_los


VGG16
Epoch 1/70
84/84 - 23s - 276ms/step - accuracy: 0.2351 - loss: 2.2552 - val_accuracy: 0.3393 - val_loss: 1.7465
Epoch 2/70
84/84 - 15s - 180ms/step - accuracy: 0.3449 - loss: 1.7301 - val_accuracy: 0.4174 - val_loss: 1.5530
Epoch 3/70
84/84 - 15s - 182ms/step - accuracy: 0.4029 - loss: 1.5623 - val_accuracy: 0.4814 - val_loss: 1.4065
Epoch 4/70
84/84 - 15s - 183ms/step - accuracy: 0.4524 - loss: 1.4200 - val_accuracy: 0.5208 - val_loss: 1.2818
Epoch 5/70
84/84 - 16s - 185ms/step - accuracy: 0.4724 - loss: 1.3442 - val_accuracy: 0.5201 - val_loss: 1.2368
Epoch 6/70
84/84 - 15s - 184ms/step - accuracy: 0.4930 - loss: 1.2770 - val_accuracy: 0.5469 - val_loss: 1.2146
Epoch 7/70
84/84 - 15s - 183ms/step - accuracy: 0.5152 - loss: 1.2249 - val_accuracy: 0.5439 - val_loss: 1.1586
Epoch 8/70
84/84 - 15s - 183ms/step - accuracy: 0.5332 - loss: 1.1850 - val_accuracy: 0.5826 - val_loss: 1.1122
Epoch 9/70
84/84 - 15s - 182ms/step - accuracy: 0.5554 - loss: 1.1506 - val_accuracy: 0.5952 - va

         Model  Test Accuracy  Training Time (s)
0  InceptionV3       0.755952         104.221189
1     Xception       0.776786         150.843951
2    MobileNet       0.771429          60.885965
3  MobileNetV2       0.786905          77.086972
4  DenseNet121       0.820833         188.740318
5        VGG16       0.679167         627.622063


In [ ]:
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Concatenate

def combined_cnn_model(image_shape=(224, 224, 3), meta_shape=(10,), num_classes=10):
    image_input = Input(shape=image_shape, name="image_input")
    x = Conv2D(32, (3, 3), activation='relu')(image_input)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)

    meta_input = Input(shape=meta_shape, name="meta_input")
    m = Dense(64, activation='relu')(meta_input)
    m = Dropout(0.3)(m)

    combined = Concatenate()([x, m])
    combined = Dense(128, activation='relu')(combined)
    combined = Dropout(0.5)(combined)
    output = Dense(num_classes, activation='softmax')(combined)

    model = Model(inputs=[image_input, meta_input], outputs=output)

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model
model = combined_cnn_model(image_shape=X_img_train.shape[1:], meta_shape=X_meta_train.shape[1:], num_classes=len(le.classes_))

history = model.fit(
    [X_img_train, X_meta_train], y_train_enc,
    epochs=70,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping]
)

y_prob = model.predict([X_img_test, X_meta_test])
y_pred = np.argmax(y_prob, axis=1)
y_true = np.argmax(y_test_enc, axis=1)



Epoch 1/70
168/168 ━━━━━━━━━━━━━━━━━━━━ 19s 70ms/step - accuracy: 0.1386 - loss: 2.7881 - val_accuracy: 0.2999 - val_loss: 1.9366
Epoch 2/70
168/168 ━━━━━━━━━━━━━━━━━━━━ 7s 40ms/step - accuracy: 0.2496 - loss: 2.0178 - val_accuracy: 0.3705 - val_loss: 1.6708
Epoch 3/70
168/168 ━━━━━━━━━━━━━━━━━━━━ 7s 40ms/step - accuracy: 0.3341 - loss: 1.7765 - val_accuracy: 0.3735 - val_loss: 1.6031
Epoch 4/70
168/168 ━━━━━━━━━━━━━━━━━━━━ 7s 40ms/step - accuracy: 0.3542 - loss: 1.6903 - val_accuracy: 0.3914 - val_loss: 1.5563
Epoch 5/70
168/168 ━━━━━━━━━━━━━━━━━━━━ 7s 40ms/step - accuracy: 0.3762 - loss: 1.6228 - val_accuracy: 0.4152 - val_loss: 1.4805
Epoch 6/70
168/168 ━━━━━━━━━━━━━━━━━━━━ 7s 40ms/step - accuracy: 0.3914 - loss: 1.5410 - val_accuracy: 0.4561 - val_loss: 1.4620
Epoch 7/70
168/168 ━━━━━━━━━━━━━━━━━━━━ 7s 40ms/step - accuracy: 0.4321 - loss: 1.4899 - val_accuracy: 0.4747 - val_loss: 1.3672
Epoch 8/70
168/168 ━━━━━━━━━━━━━━━━━━━━ 7s 39ms/step - accuracy: 0.4639 - loss: 1.4179 - val_acc

In [ ]:
param = "meta"
name = "my"
loss, acc = model.evaluate([X_img_test, X_meta_test], y_test_enc)

print("\nClassification Report:")
print(classification_report(y_true, y_pred))

with open(f"/content/drive/MyDrive/Master - A/3_fig/{param}_{name}_results.txt", "w") as f:
    f.write(f"Model: {name}\n")
    #f.write(f"Training time: {training_time:.2f} sec\n")
    f.write(f"Accuracy: {acc:.4f}\n\n")
    f.write("Classification Report:\n")
    f.write(classification_report(y_true, y_pred))

conf_matrix = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=le.classes_, yticklabels=le.classes_)
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.tight_layout()
plt.savefig(f"/content/drive/MyDrive/Master - A/3_fig/{param}_{name}_matrix.png")
plt.close()

plt.figure(figsize=(8, 6))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.tight_layout()
plt.savefig(f"/content/drive/MyDrive/Master - A/3_fig/{param}_{name}_train_val_loss.png")
plt.close()

plt.figure(figsize=(8, 6))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.tight_layout()
plt.savefig(f"/content/drive/MyDrive/Master - A/3_fig/{param}_{name}_train_val_accuracy.png")
plt.close()

fpr, tpr, _ = roc_curve(y_true, y_prob[:, 1]) if y_prob.shape[1] == 2 else (None, None, None)
if fpr is not None:
    roc_auc = auc(fpr, tpr)
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.2f})', lw=2)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"/content/drive/MyDrive/Master - A/3_fig/{param}_{name}_roc_curve.png")
    plt.close()

model.save(f'/content/drive/MyDrive/Master - A/3_fig/{param}_{name}_model.h5')

results.append({
    'Model': name,
    'Test Accuracy': acc,
    'Training Time (s)': training_time
})

53/53 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.5904 - loss: 1.0868

Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.77      0.82       192
           1       0.46      0.74      0.57       158
           2       0.81      0.88      0.84       161
           3       0.68      0.60      0.64       149
           4       0.83      0.69      0.76       170
           5       0.48      0.56      0.52       164
           6       0.64      0.26      0.37       181
           7       0.54      0.71      0.61       184
           8       0.44      0.45      0.45       169
           9       0.49      0.43      0.46       152

    accuracy                           0.61      1680
   macro avg       0.63      0.61      0.60      1680
weighted avg       0.63      0.61      0.61      1680



In [ ]:
results_df = pd.DataFrame(results)
results_df.to_csv(f'/content/drive/MyDrive/Master - A/3_fig/{param}_model_training_results.csv', index=False)
print(results_df)

         Model  Test Accuracy  Training Time (s)
0  InceptionV3       0.738690         127.865819
1     Xception       0.760119         208.671374
2    MobileNet       0.792857          89.317813
3  MobileNetV2       0.794048          99.906089
4  DenseNet121       0.818452         209.636113
5        VGG16       0.675000         891.858847
6           my       0.551190          91.106546
7           my       0.610119          91.106546


## Klasyczne

In [10]:
from sklearn.metrics import classification_report, accuracy_score,confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import csv

In [8]:
X_img_train_flat = X_img_train.reshape(X_img_train.shape[0], -1)
X_img_test_flat = X_img_test.reshape(X_img_test.shape[0], -1)

In [5]:
X_img_train_flat = X_img_train.reshape(X_img_train.shape[0], -1)
X_img_test_flat = X_img_test.reshape(X_img_test.shape[0], -1)

from sklearn.decomposition import PCA

pca = PCA(n_components=2000)
X_train_pca = pca.fit_transform(X_img_train_flat)
X_test_pca = pca.transform(X_img_test_flat)
X_train_combined = np.concatenate([X_train_pca, X_meta_train], axis=1)
X_test_combined = np.concatenate([X_test_pca, X_meta_test], axis=1)


In [ ]:
del X_meta_train, X_meta_test,

In [9]:
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn import svm
from sklearn.linear_model import LogisticRegression
import time

In [11]:
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import load_model

In [ ]:
import time
import numpy as np
import gc
import csv
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix, roc_curve, auc
)
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB


model_classes = [
    ('LogisticRegression_D', LogisticRegression(multi_class='multinomial', max_iter=5000)),
    #('NaiveBayes_D', GaussianNB())
]

results = []
param = "meta22"

y_train_labels = np.argmax(y_train_enc, axis=1)
y_test_labels = np.argmax(y_test_enc, axis=1)

for name, model in model_classes:
    print(f"\n{name} - training...")
    start_time = time.time()

    model.fit(X_train_combined, y_train_labels)
    end_time = time.time()
    training_time = end_time - start_time

    y_pred = model.predict(X_test_combined)

    try:
        y_prob = model.predict_proba(X_test_combined)
    except:
        y_prob = None

    accuracy = accuracy_score(y_test_labels, y_pred)
    report = classification_report(y_test_labels, y_pred)

    with open(f"/content/drive/MyDrive/Master - A/3_fig/{param}_{name}_results.txt", "w") as f:
        f.write(f"Model: {name}\n")
        f.write(f"Training time: {training_time:.2f} sec\n")
        f.write(f"Accuracy: {accuracy:.4f}\n\n")
        f.write("Classification Report:\n")
        f.write(report)

    conf_matrix = confusion_matrix(y_test_labels, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues",
                xticklabels=le.classes_, yticklabels=le.classes_)
    plt.title("Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.tight_layout()
    plt.savefig(f"/content/drive/MyDrive/Master - A/3_fig/{param}_{name}_matrix.png")
    plt.close()

    if y_prob is not None and y_prob.shape[1] == 2:
        fpr, tpr, _ = roc_curve(y_test_labels, y_prob[:, 1])
        roc_auc = auc(fpr, tpr)
        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.2f})', lw=2)
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title("ROC Curve")
        plt.legend()
        plt.tight_layout()
        plt.savefig(f"/content/drive/MyDrive/Master - A/3_fig/{param}_{name}_roc_curve.png")
        plt.close()

    results.append((name, accuracy))

    del model, y_pred
    gc.collect()

with open(f"/content/drive/MyDrive/Master - A/3_fig/{param}_final_results.csv", mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Model", "Accuracy"])
    for name, acc in results:
        writer.writerow([name, f"{acc:.4f}"])

print("\nFinal Results:")
for name, acc in results:
    print(f"{name}: {acc:.4f}")



LogisticRegression_D - training...


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(



Final Results:
LogisticRegression_D: 0.3696


In [ ]:
col_means = np.nanmean(X_meta_test, axis=0)
inds = np.where(np.isnan(X_meta_test))
X_meta_test[inds] = np.take(col_means, inds[1])

col_means = np.nanmean(X_meta_train, axis=0)
inds = np.where(np.isnan(X_meta_train))
X_meta_train[inds] = np.take(col_means, inds[1])

X_img_train_flat = X_img_train.reshape(X_img_train.shape[0], -1)
X_img_test_flat = X_img_test.reshape(X_img_test.shape[0], -1)
X_train_combined = np.concatenate([X_img_train_flat, X_meta_train], axis=1)
X_test_combined = np.concatenate([X_img_test_flat, X_meta_test], axis=1)

In [19]:
X_train_combined.shape, X_test_combined.shape

((6717, 150533), (1680, 150533))

In [ ]:
import numpy as np

col_means = np.nanmean(X_meta_test, axis=0)
inds = np.where(np.isnan(X_meta_test))
X_meta_test[inds] = np.take(col_means, inds[1])

col_means = np.nanmean(X_meta_train, axis=0)
inds = np.where(np.isnan(X_meta_train))
X_meta_train[inds] = np.take(col_means, inds[1])



In [ ]:
import gc
model_classes = [
    ('KNN_D', KNeighborsClassifier()),
    ('DecisionTree_D', DecisionTreeClassifier()),
    ('RandomForest_D', RandomForestClassifier()),
    #('LogisticRegression_D', LogisticRegression()),
    #('NaiveBayes_D', GaussianNB())
]

results = []
param = "5_meta"

for name, model in model_classes:
    print(f"\n{name} - training...")
    start_time = time.time()

    model.fit(X_train_combined, y_train_enc)
    end_time = time.time()
    training_time = end_time - start_time

    preds = model.predict(X_test_combined)
    acc_preview = accuracy_score(y_test_enc, preds)

    y_prob = model.predict(X_test_combined)
    y_pred = np.argmax(y_prob, axis=1)
    #y_pred = y_prob
    y_true = np.argmax(y_test_enc, axis=1)

    accuracy = accuracy_score(y_true, y_pred)
    report = classification_report(y_true, y_pred)

    with open(f"/content/drive/MyDrive/Master - A/3_fig/{param}_{name}_results.txt", "w") as f:
        f.write(f"Model: {name}\n")
        f.write(f"Training time: {training_time:.2f} sec\n")
        f.write(f"Accuracy: {accuracy:.4f}\n\n")
        f.write("Classification Report:\n")
        f.write(report)

    conf_matrix = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=le.classes_, yticklabels=le.classes_)
    plt.title("Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.tight_layout()
    plt.savefig(f"/content/drive/MyDrive/Master - A/3_fig/{param}_{name}_matrix.png")
    plt.close()

    fpr, tpr, _ = roc_curve(y_true, y_prob[:, 1]) if y_prob.shape[1] == 2 else (None, None, None)
    if fpr is not None:
        roc_auc = auc(fpr, tpr)
        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.2f})', lw=2)
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.legend()
        plt.tight_layout()
        plt.savefig(f"/content/drive/MyDrive/Master - A/3_fig/{param}_{name}_roc_curve.png")
        plt.close()

    results.append((name, accuracy))

    del model, preds
    gc.collect()

with open(f"/content/drive/MyDrive/Master - A/3_fig/{param}_final_results.csv", mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Model", "Accuracy"])
    for name, acc in results:
        writer.writerow([name, f"{acc:.4f}"])

print("\nFinal Results:")
for name, acc in results:
    print(f"{name}: {acc:.4f}")



KNN_D - training...

DecisionTree_D - training...

RandomForest_D - training...


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Final Results:
KNN_D: 0.1875
DecisionTree_D: 0.3333
RandomForest_D: 0.1345


In [ ]:
model_classes = [
    ('KNN_D', KNeighborsClassifier()),
    ('DecisionTree_D', DecisionTreeClassifier()),
     ('RandomForest_D', RandomForestClassifier()),
    #('LogisticRegression_D', LogisticRegression(multi_class='multinomial')),
    #('NaiveBayes_D', GaussianNB())
]

results = []
param = "pca_meta2"

for name, model in model_classes:
    print(f"\n{name} - training...")
    start_time = time.time()

    y_train_labels = np.argmax(y_train_enc, axis=1)
    y_test_labels = np.argmax(y_train_enc, axis=1)
    model.fit(X_train_combined, y_train_labels)
    end_time = time.time()
    training_time = end_time - start_time

    preds = model.predict(X_test_combined)

    y_prob = model.predict(X_test_combined)
    y_pred = y_prob
    y_true = np.argmax(y_test_enc, axis=1)
    acc_preview = accuracy_score(y_test_labels, y_pred)


    accuracy = accuracy_score(y_true, y_pred)
    report = classification_report(y_true, y_pred)

    with open(f"/content/drive/MyDrive/Master - A/3_fig/{param}_{name}_results.txt", "w") as f:
        f.write(f"Model: {name}\n")
        f.write(f"Training time: {training_time:.2f} sec\n")
        f.write(f"Accuracy: {accuracy:.4f}\n\n")
        f.write("Classification Report:\n")
        f.write(report)

    conf_matrix = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=le.classes_, yticklabels=le.classes_)
    plt.title("Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.tight_layout()
    plt.savefig(f"/content/drive/MyDrive/Master - A/3_fig/{param}_{name}_matrix.png")
    plt.close()

    fpr, tpr, _ = roc_curve(y_true, y_prob[:, 1]) if y_prob.shape[1] == 2 else (None, None, None)
    if fpr is not None:
        roc_auc = auc(fpr, tpr)
        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.2f})', lw=2)
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.legend()
        plt.tight_layout()
        plt.savefig(f"/content/drive/MyDrive/Master - A/3_fig/{param}_{name}_roc_curve.png")
        plt.close()

    results.append((name, accuracy))

    del model, preds
    gc.collect()

with open(f"/content/drive/MyDrive/Master - A/3_fig/{param}_final_results.csv", mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Model", "Accuracy"])
    for name, acc in results:
        writer.writerow([name, f"{acc:.4f}"])

print("\nFinal Results:")
for name, acc in results:
    print(f"{name}: {acc:.4f}")


KNN_D - training...


ValueError: Found input variables with inconsistent numbers of samples: [6717, 1680]

In [ ]:

model_classes = [
    ('LogisticRegression_D', LogisticRegression(multi_class='multinomial', max_iter=5000)),
    ('NaiveBayes_D', GaussianNB())
]

results = []
param = "55_z_meta22"

y_train_labels = np.argmax(y_train_enc, axis=1)
y_test_labels = np.argmax(y_test_enc, axis=1)

for name, model in model_classes:
    print(f"\n{name} - training...")
    start_time = time.time()
    print(start_time)
    model.fit(X_train_combined, y_train_labels)
    end_time = time.time()
    training_time = end_time - start_time

    y_pred = model.predict(X_test_combined)

    try:
        y_prob = model.predict_proba(X_train_combined)
    except:
        y_prob = None

    accuracy = accuracy_score(y_test_labels, y_pred)
    report = classification_report(y_test_labels, y_pred)

    with open(f"/content/drive/MyDrive/Master - A/3_fig/{param}_{name}_results.txt", "w") as f:
        f.write(f"Model: {name}\n")
        f.write(f"Training time: {training_time:.2f} sec\n")
        f.write(f"Accuracy: {accuracy:.4f}\n\n")
        f.write("Classification Report:\n")
        f.write(report)

    conf_matrix = confusion_matrix(y_test_labels, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues",
                xticklabels=le.classes_, yticklabels=le.classes_)
    plt.title("Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.tight_layout()
    plt.savefig(f"/content/drive/MyDrive/Master - A/3_fig/{param}_{name}_matrix.png")
    plt.close()

    if y_prob is not None and y_prob.shape[1] == 2:
        fpr, tpr, _ = roc_curve(y_test_labels, y_prob[:, 1])
        roc_auc = auc(fpr, tpr)
        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.2f})', lw=2)
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title("ROC Curve")
        plt.legend()
        plt.tight_layout()
        plt.savefig(f"/content/drive/MyDrive/Master - A/3_fig/{param}_{name}_roc_curve.png")
        plt.close()

    results.append((name, accuracy))

    del model, y_pred
    gc.collect()

with open(f"/content/drive/MyDrive/Master - A/3_fig/{param}_final_results.csv", mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Model", "Accuracy"])
    for name, acc in results:
        writer.writerow([name, f"{acc:.4f}"])

print("\nFinal Results:")
for name, acc in results:
    print(f"{name}: {acc:.4f}")



LogisticRegression_D - training...
1749222058.6759188


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



NaiveBayes_D - training...
1749229691.2187786

Final Results:
LogisticRegression_D: 0.3851
NaiveBayes_D: 0.3095
